<a href="https://colab.research.google.com/github/Thomascc123/Proyecto-Kaggle-Modelos/blob/master/99%20-%20modelo%20soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>